# Hierarchical COCO128 example

This example explores training a hierarchical model on the [COCO128 dataset](https://docs.ultralytics.com/datasets/detect/coco128/), which is a very small subset of the
classic [COCO dataset](https://cocodataset.org).  COCO128 is mostly intended as a "sanity check" dataset to quickly make sure that a model architecture and training routine are
not utterly broken before embarking on a longer and more expensive training over a larger dataset.

This example assumes some familiarity with the ultralytics family of YOLO models.  If you have never used these, you had ought to at least explore the [quickstart](https://docs.ultralytics.com/quickstart/) and the [ultralytics COCO128 example](https://docs.ultralytics.com/datasets/detect/coco128/).

This example does not assume that you have any familiarity with convolutional neural networks or the mathematics of probability or MLOps or methodologies for hyperparameter selection.  However, these topics would be useful if you actually intend to use the hierarchical model outside of this toy example.

## Setup

In [1]:
from ultralytics import YOLO
import os
from hierarchical_yolo.mscoco_hierarchical_model import MSCOCOHierarchicalDetectionTrainer
import hierarchical_yolo.yolo_utils
import hierarchical_loss.path_utils
import hierarchical_loss.viz_utils
import ultralytics
import torch
from PIL import Image

### Data and Configs

Ultralytics YOLO requires multiple configuration and weight files, and the default storage location for the trained models (runs) are not always so great.
You should inspect these various paths to ensure that they align with your data management strategy.

In [2]:
# Ultralytics default settings.  See https://docs.ultralytics.com/quickstart/#ultralytics-settings
# The root directory where datasets are expected to be stored, and where ultralytics will download curated datasets
DATASETS = ultralytics.settings['datasets_dir']
# The root directory for model weights.  In particular, where ultralytics will download pre-trained model .pt weight files
WEIGHTS = ultralytics.settings['weights_dir']
# The root directory where training runs will be stored, including evaluation artifacts and trained model .pt weight files
RUNS = ultralytics.settings['runs_dir']

# The pre-trained model to use.  Note the v8 (version 8) and the n (nano-scale)
YOLO_BASE_WEIGHTS = os.path.join(WEIGHTS, "yolov8n.pt")

# Configure project runs save locations.
BASE_PROJECT = os.path.join(RUNS, 'coco128')
HIERARCHICAL_PROJECT = os.path.join(RUNS, 'hierarchical_coco128')

# Find where data is downloaded/stored.  This is where ultralytics will download the coco128 data.
DATA = os.path.join(DATASETS, 'coco128')
IMAGES_PATH = os.path.join(DATA, 'images', 'train2017')

# Find model configurations.  These are in the repository https://github.com/csbrown-noaa/hierarchical_yolo.
# If you aren't running this from the cloned repo, you will need to go acquire these and change MODEL_CONFIGS to reflect the location of the model config files.
MODEL_CONFIGS = '../hierarchical_yolo/models'
COCO128_MODELS = os.path.join(MODEL_CONFIGS, 'coco128')
YOLO_MODEL_YAML = os.path.join(COCO128_MODELS, 'hierarchicalcoco128yolov8.yaml')
YOLO_DATASET_YAML = os.path.join(COCO128_MODELS, 'hierarchicalcoco128.yaml')

### Run on GPU or CPU

If a GPU is available, we can use it.

In [3]:
devices = list(range(torch.cuda.device_count()))
devices = devices[0] if devices else 'cpu'
devices

0

## "Regular" YOLO model

If you don't already have the pre-trained weight file and the coco128 dataset, this will automatically download them, so you probably want to run this at least once.
You can make this cell runnable by selecting it and pressing the "y" key.  

This is also useful for comparison purposes.  You can compare predictions from this model with ones from the hierarchical model below.  However...

***NB:*** The ultralytics YOLO models are pre-trained *on the COCO dataset*.  So, "evaluating" the performance of the model, even informally, on COCO or COCO-derived datasets is not sound.  As such, any comparisons should probably be limited to the format of predictions and how the hierarchical predictions are arranged vs. the "flat" predictions.

## Hierarchical YOLO model

### The Hierarchy

The original COCO dataset does not include a hierarchical representation.  We have formulated one for this example.  It is displayed below.  Note that all of the leaf nodes are original COCO categories.

***Technical Details for implementing your own hierarchy:*** If you want to train an ultralytics YOLO detection model on your own custom data, you will need to define the categories and the hierarchy.  

For ultralytics YOLO, the categories are provided in the training data configuration yaml.  You can see this in action with [the original COCO128 yaml](https://github.com/csbrown-noaa/hierarchical_yolo/blob/main/hierarchical_yolo/models/coco128/coco128.yaml). The "extended" categories that include our custom "higher-level" categories are defined in the dataset yaml files that we define above, and which can be found [here](https://github.com/csbrown-noaa/hierarchical_yolo/blob/main/hierarchical_yolo/models/coco128/hierarchicalcoco128.yaml) and [here](https://github.com/csbrown-noaa/hierarchical_yolo/blob/main/hierarchical_yolo/models/coco128/hierarchicalcoco128yolov8.yaml).  Note the `nc` and `names` parameters in these files.  

The hierarchy is defined as a `_hierarchy` property of [the `MSCOCOHierarchicalDetectionTrainer` class](https://github.com/csbrown-noaa/hierarchical_yolo/blob/main/hierarchical_yolo/mscoco_hierarchical_model.py).    However, these yaml files do not encode the actual hierarchical relationship between the categories.  The user must do this by extending the [`HierarchicalDetectionTrainer` class](https://github.com/csbrown-noaa/hierarchical_yolo/blob/main/hierarchical_yolo/hierarchical_detection.py) and providing a `_hierarchy` class property.  The [`MSCOCOHierarchicalDetectionTrainer` class](https://github.com/csbrown-noaa/hierarchical_yolo/blob/main/hierarchical_yolo/mscoco_hierarchical_model.py) provides a good example of how to accomplish this.  

***Technical details for implementing your own custom loss:*** The `MSCOCOHierarchicalDetectionTrainer` class utilizes the custom loss and training classes defined in [the `hierarchical_yolo.hierarchical_detection` module](https://github.com/csbrown-noaa/hierarchical_yolo/blob/main/hierarchical_yolo/hierarchical_detection.py).  In order to extend this functionality to, say, segmentation, new loss and training classes would need to be defined.  Although these may look daunting, in reality, 99% of the code is copy-pasta from [the ultralytics implementations](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/utils/loss.py#L194).  In particular, the "new" code is limited to propagating the `hierarchy` down the call stack to the loss function and replacing a few lines for the class loss to utilize the [`hierarchical_loss` library](https://github.com/csbrown-noaa/hierarchical_loss).  The existing bounding box loss and dfl loss remain unchanged.

In [4]:
hierarchical_loss.viz_utils.viz_tree(MSCOCOHierarchicalDetectionTrainer._hierarchy.raw_tree).show()

object : 
├── accessory_and_item : 
│   ├── home_and_office_item : 
│   │   ├── book : 
│   │   ├── clock : 
│   │   ├── scissors : 
│   │   ├── teddy bear : 
│   │   └── vase : 
│   └── personal_accessory : 
│       ├── backpack : 
│       ├── handbag : 
│       ├── suitcase : 
│       ├── tie : 
│       └── umbrella : 
├── food : 
│   ├── dessert : 
│   │   ├── cake : 
│   │   └── donut : 
│   ├── fruit : 
│   │   ├── apple : 
│   │   ├── banana : 
│   │   └── orange : 
│   ├── prepared_meal : 
│   │   ├── hot dog : 
│   │   ├── pizza : 
│   │   └── sandwich : 
│   └── vegetable : 
│       ├── broccoli : 
│       └── carrot : 
├── indoor_object : 
│   ├── appliance_and_electronics : 
│   │   ├── appliance : 
│   │   │   ├── microwave : 
│   │   │   ├── oven : 
│   │   │   ├── refrigerator : 
│   │   │   ├── sink : 
│   │   │   ├── toaster : 
│   │   │   └── toilet : 
│   │   └── electronics : 
│   │       ├── cell phone : 
│   │       ├── keyboard : 
│   │       ├── laptop : 
│   │  

### Model training

Note that loading the model is identical to the "basic" YOLO model.

The differences to note are: 

1) utilizing the dataset yaml file (YOLO_MODEL_YAML) that includes the "extended" COCO categories that include our custom higher-level categories as seen above.  [See above](#The-Hierarchy) for more details.
2) providing a custom `trainer` to the `train` method.  This custom trainer implements our hierarchy and the custom loss function that utilizes the hierarchy.  [See above](#The-Hierarchy) for more details.

In [5]:
hierarchical_model = YOLO(YOLO_MODEL_YAML).load(YOLO_BASE_WEIGHTS)  # build a new model from scratch

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Transferred 319/355 items from pretrained weights


In [ ]:
results = hierarchical_model.train(
    data=YOLO_DATASET_YAML,
    project=HIERARCHICAL_PROJECT,
    epochs=100,
    imgsz=640, 
    box=20, #upweight the box loss.  The marginal confidences tend to be lower for the hierarchical case, which makes the cls_loss higher, on average.
    trainer=MSCOCOHierarchicalDetectionTrainer,
    device=devices,
    val=False
)

New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.186 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=20, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../hierarchical_yolo/models/coco128/hierarchicalcoco128.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../hierarchical_yolo/models/coco128/hierarchicalcoco128yolov8.yaml, m

### Model Results

Now that we've trained a model, we can inspect the results.  Evaluating model performance with hierarchical categories does not fit with the usual paradigms for evaluating model performance, so we can't naively use [the existing ultralytics model evaluation metrics](https://docs.ultralytics.com/reference/utils/metrics/).  There exist various mechanisms for evaluating performance of hierarchical models, [e.g. here](https://www.kaggle.com/competitions/fathomnet-2025).  Also possible are casting the hierarchical problem into a "flat" problem by only considering the leaf nodes or the root nodes and comparing with a bespoke model trained on only those categories.  However, we do not perform any of this here, and limit ourselves to visualizing the results.

In [ ]:
# Find the latest model from the last training run for this project
trained_models = os.listdir(HIERARCHICAL_PROJECT)
model_numbers = map(lambda x: int(x[len('train'):]) if len(x) > len('train') else 0, trained_models)
latest_model = 'train' + str(max(model_numbers))
latest_model

In [ ]:
# Load up the weights from the last training run in evaluation mode
trained_model = (
    YOLO(os.path.join(HIERARCHICAL_PROJECT, latest_model, 'weights', 'best.pt')).to(devices)
)
trained_model.eval()

In [ ]:
# load in a test image from the training dataset.
image_path = os.path.join(IMAGES_PATH, '000000000257.jpg')
image = Image.open(image_path).convert('RGB')
image

#### Raw Predictions

Probably, you won't need these.  However, we include them here to illustrate how we interpret the hierarchical predictions. 

Note the shape of these.  The first value in the shape, the number of rows, is 4 + number_of_categories.  The first 4 indices are the bounding box corners, and the remaining 109 are the class confidences.  There are 109 classes, including the 80 original COCO categories, and 29 additional "higher-level" categories, all of which you can see in the class tree above.  The second value, the number of columns, is the number of bounding box predictions made.  This is a fixed value based on our model architecture.  Typically, we filter these heavily due to large amounts of overlap, etc, to get only a handful of predictions using a method called [non maximum supression](https://medium.com/analytics-vidhya/non-max-suppression-nms-6623e6572536).  We will investigate this later.

In [ ]:
raw_predictions = hierarchical_yolo.yolo_utils.yolo_raw_predict(trained_model, [image], (640, 640))[0]
raw_predictions.shape

In [ ]:
# Get the raw predictions for a single bounding box proposal
ROOT_NODE_IDX = int(MSCOCOHierarchicalDetectionTrainer._hierarchy.roots)
N_BBOX_INDICES = 4
# This gives the index of the object with the highest confidence for the root "object" class
one_full_pred_loc = int(torch.argmax(raw_predictions[N_BBOX_INDICES + ROOT_NODE_IDX]))
one_full_obj_preds = raw_predictions[N_BBOX_INDICES:, one_full_pred_loc]
one_full_obj_preds

In [ ]:
hierarchical_loss.viz_utils.viz_tree(
    MSCOCOHierarchicalDetectionTrainer._hierarchy.index_tree, 
    name_map = MSCOCOHierarchicalDetectionTrainer._hierarchy.idx_to_node, 
    vals = list(map(lambda x: '{:.4f}'.format(x), one_full_obj_preds))
).show()

#### Interpreting Hierarchical Predictions

The way that the model is trained computes a class loss that interprets the raw predictions as *conditional probabilities*.  This means that the raw score corresponding to, say, "train" is not the model's confidence that the object is a train.  It is the model's confidence that the object is a train **GIVEN** that the object is a "land_vehicle".  The raw score corresponding to "land_vehicle", in turn, is conditioned on "vehicle", which is conditioned on "object".  So, if you look through the leaves of the scores above, you may notice very high confidences reported for nodes whose parentage actually have low scores.  These are actually interpreted by the model to be "unlikely", since the score is only conditional.  One can obtain the marginal confidences by multiplying down a branch:

If $C_0, C_1, C_2, \ldots, C_n$ are a branch of nodes in a hierarchy such that belonging to a leaf class, $C_n$ necessarily implies belonging to the parent class $C_{n-1}$, and so on up to the root node $C_0$, then the marginal confidence can be recovered as:

$$ P(C_n) = P(C_n | C_{n-1}) * P(C_{n-1} | C_{n-2}) * \ldots * P(C_1 | C_0) * P(C_0) $$

The confidence for root nodes is not conditional, since there is nothing left to condition on.

#### Filtered Predictions

The default filtering methods provided [as methods on the ultralytics YOLO model classes](https://docs.ultralytics.com/modes/predict/#key-features-of-predict-mode) perform filtering as thought the confidences are *marginal*.  Since we are interpreting our confidences as *conditional*, then these methods do not work correctly for our hierarchical output.  Therefore, we provide several convenience methods for interpreting the raw output.

We employ the [non maximum supression provided by ultralytics](https://docs.ultralytics.com/reference/utils/nms/) on only the root categories to filter the results.  Furthermore, we filter the raw confidences across all categories to only choose an "optimal path", which essentially chases down the maximum marginal confidence at each layer in the hierarchy, beginning with the root.  Note that the first value in the first prediction is the index of the root node.

In [ ]:
predicted_boxes, predicted_paths, predicted_path_scores = hierarchical_yolo.yolo_utils.hierarchical_predict(trained_model, MSCOCOHierarchicalDetectionTrainer._hierarchy.index_tree, [image])
predicted_paths[0][0]

In [ ]:
int(MSCOCOHierarchicalDetectionTrainer._hierarchy.roots)

#### More Filtering

The `hierarchical_predict` method follows the prediction all the way from the root to a leaf.  We can further filter this by truncating the path once we reach a conditional confidence that is below a certain threshold.  Depending on the threshold chosen, the truncation results may not even include the root node, so we further filter these empty paths.

In [ ]:
tpaths, tscores = hierarchical_loss.path_utils.truncate_paths_marginals(predicted_paths[0], predicted_path_scores[0], threshold=0.2)

In [ ]:
fboxes, fpaths, fscores = hierarchical_loss.path_utils.filter_empty_paths(predicted_boxes[0], tpaths, tscores)
fboxes.shape, len(fpaths), len(fscores)

#### Visualizing the results

Here, we draw the filtered predictions on the image, along with the furthest-down-the-hierarchy predicted category and the marginal confidence of that category.

In [ ]:
marginal_confidences = [float(torch.prod(fscore)) for fscore in fscores]
lowest_category_in_path = [MSCOCOHierarchicalDetectionTrainer._hierarchy.idx_to_node[fpath[-1]] for fpath in fpaths]
hierarchical_loss.viz_utils.draw_boxes_on_image(image, fboxes.T, 
                    labels=lowest_category_in_path, 
                    scores=marginal_confidences, 
                    box_color=(0, 255, 0), text_color=(255, 255, 255))